In [ ]:
import sys
import json
from pathlib import Path
from google.oauth2.credentials import Credentials


root_dir = Path("/workspaces/DeliveryTracker")

sys.path.insert(0, str(root_dir))


credential_json = json.load((root_dir / "google_token/logged_in_credentials.json").open())
credentials = Credentials.from_authorized_user_info(credential_json)

In [ ]:
import logging
from shared.GoogleServices import GmailService
from shared.GoogleServices.GmailQueryBuilder import GmailQueryBuilder


gmail_service = GmailService(credentials)

query = (
    GmailQueryBuilder()
    .from_email("personal@relaxdays.de")
    .subject("Lohnschein/Korrekturlohnschein")
    .has_attachment()
    .build()
)

message_ids = [message["id"] for message in gmail_service._query_messages(query)]

for msg_id in message_ids:
    file_paths = [
        Path(path) for path in gmail_service.download_pdf_attachments(msg_id)
    ]

    logging.info(f"Downloaded {file_paths} attachments from message {msg_id}")

    for file_path in file_paths:
        file_path.unlink()

    break